In [1]:
#!/usr/bin/python3
import math
import json

# For each context, prior over the states
prior_states = eval(open('irony/synthetic_prior_states.json', 'r').read())
print("prior states: ", prior_states)
# For each state, prior over valence+arousal combination
prior_affect = eval(open('irony/prior_affect.json', 'r').read())

# Prior over QUDs
# hmmm was this ever calculated in the paper?
prior_quds = {
    "state":    0.3,
    "valence":  0.3,
    "arousal":  0.4,
}


states = [1,2,3,4,5]
utterances = [1,2,3,4,5]
quds = [ "state", "valence", "arousal"]
valences = ["positive", "negative"]
arousals = ["high", "low"]
affects = [(v, a) for v in valences for a in arousals]
contexts = list(prior_states.keys())

rationality_factor = 1.0

# the q function in the paper
# 's' is the state of the world, 'A' reps the speaker's affect toward the state
# this serves as a projection from full meaning spact to subset of speaker's interest
def qud(q, s, A):
    if q == "state": return s
    if q == "valence": return A[0]
    if q == "arousal": return A[1]
    print("error")


def literal_listener(s, A, u):
    if s != u:          #if state does not == utterance. This is the basic RSA that returns priors for true states.
        return 0.0
    else:
        return prior_affect[s][A] #so the literal listener will return literally whatever the qud asks for. but what happens if the qud is the state? should we return the probability of a weather state happening? If it doesn't it probs won't change behavior because they're all uniform.


# the U function in the paper, without the log
# this models info gained by listener about topic of interest
# where q is the intended QUD

def exp_utility(u, s, A, q):
    sum = 0.0
    for sp in states:
        for Ap in affects:
            if qud(q, s, A) == qud(q, sp, Ap):
                sum += literal_listener(sp, Ap, u)
    return sum

# the S function in the paper, normalized
# "the speaker S chooses an utterance according to a softmax decision rule"
# in the paper, it says the base is e. How come it's the the utility ftn^ rationality factior here?
def speaker(u, s, A, q):
    norm = 0.0
    for up in utterances:
        norm += math.pow(exp_utility(up, s, A, q), rationality_factor)
    return math.pow(exp_utility(u, s, A, q), rationality_factor) / norm

# the pragmatic L function in the paper, unnormalized
# this is the ftn that models the ambiguity behind QUD, particularly it's held in the sum
def unnorm_pragmatic_listener(s, A, u, context):
    sum = 0.0
    for q in quds:
        sum += prior_quds[q] * speaker(u, s, A, q)
    return prior_states[context][s] * prior_affect[s][A] * sum


# the pragmatic L function in the paper, normalized
def pragmatic_listener(s, A, u, context):
    norm = 0.0
    for sp in states:
        for Ap in affects:
            norm += unnorm_pragmatic_listener(sp, Ap, u, context)
    return unnorm_pragmatic_listener(s, A, u, context) / norm



prior states:  {'WC33': {1: 0.3199974844403617, 2: 0.12459021729038411, 3: 0.06120824578512506, 4: 0.32601555671818205, 5: 0.1681884957659472}, 'WC50': {1: 0.24936876761789464, 2: 0.07800146561416983, 3: 0.2857082298557959, 4: 0.25039222220969637, 5: 0.13652931470244328}, 'WC34': {1: 0.05580687401475829, 2: 0.2328182808171911, 3: 0.01703908323752032, 4: 0.31305224584908575, 5: 0.3812835160814445}, 'WC74': {1: 0.018393994666088296, 2: 0.2954197977908249, 3: 0.15717595369962556, 4: 0.29223702560948145, 5: 0.2367732282339798}, 'WC87': {1: 0.1593474999001845, 2: 0.40145588223260054, 3: 0.17507534619465973, 4: 0.07409102150386906, 5: 0.19003025016868622}, 'WC41': {1: 0.07926262040989361, 2: 0.35687669815976236, 3: 0.09417499722677371, 4: 0.3265350662555099, 5: 0.14315061794806053}, 'WC82': {1: 0.32872561903581293, 2: 0.00983942500204867, 3: 0.15034127295159405, 4: 0.16944170225670305, 5: 0.3416519807538413}, 'WC86': {1: 0.2858429630543205, 2: 0.10099760171397114, 3: 0.06026057175054682, 4: 

In [2]:
from collections import defaultdict

output = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))

def main():
    for context in contexts:
        print("----------------------")
        print("  CONTEXT: %s" % context)
        print("----------------------")
        for u in utterances:
            print("--- utterance: %s ----" % u)
            total_prob = 0.0
            for s in states:
                for A in affects:
                    prob = pragmatic_listener(s, A, u, context)
                    output[context][u][s][A] = prob
                    print("  s: %s, A: %s:\t%f" % (s, A, prob))
                    total_prob += prob
            print("  total_prob: %f" % total_prob)
main()

----------------------
  CONTEXT: WC33
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.588750
  s: 1, A: ('negative', 'low'):	0.099809
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.055363
  s: 2, A: ('negative', 'low'):	0.034203
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.000702
  s: 3, A: ('negative', 'high'):	0.003531
  s: 3, A: ('negative', 'low'):	0.025755
  s: 4, A: ('positive', 'high'):	0.039264
  s: 4, A: ('positive', 'low'):	0.018540
  s: 4, A: ('negative', 'high'):	0.009787
  s: 4, A: ('negative', 'low'):	0.041748
  s: 5, A: ('positive', 'high'):	0.073276
  s: 5, A: ('positive', 'low'):	0.004278
  s: 5, A: ('negative', 'high'):	0.004428
  s: 5, A: ('negative', 'low'):	0.000567
  total_prob: 1.000000
--- utterance: 2 ----
  s: 1, A: ('positive', 'high'):	0.000000

  s: 2, A: ('negative', 'low'):	0.009415
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.033456
  s: 3, A: ('negative', 'high'):	0.012974
  s: 3, A: ('negative', 'low'):	0.052859
  s: 4, A: ('positive', 'high'):	0.086300
  s: 4, A: ('positive', 'low'):	0.145322
  s: 4, A: ('negative', 'high'):	0.005917
  s: 4, A: ('negative', 'low'):	0.014098
  s: 5, A: ('positive', 'high'):	0.344124
  s: 5, A: ('positive', 'low'):	0.089191
  s: 5, A: ('negative', 'high'):	0.009153
  s: 5, A: ('negative', 'low'):	0.001783
  total_prob: 1.000000
----------------------
  CONTEXT: WC34
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.174642
  s: 1, A: ('negative', 'low'):	0.029606
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.175966
  s: 2, A: ('negative', 'low'):	0.108709
  s: 3, A: ('positive'

  s: 4, A: ('positive', 'high'):	0.004350
  s: 4, A: ('positive', 'low'):	0.011494
  s: 4, A: ('negative', 'high'):	0.001436
  s: 4, A: ('negative', 'low'):	0.012280
  s: 5, A: ('positive', 'high'):	0.040362
  s: 5, A: ('positive', 'low'):	0.013185
  s: 5, A: ('negative', 'high'):	0.003231
  s: 5, A: ('negative', 'low'):	0.000829
  total_prob: 1.000000
--- utterance: 3 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.062448
  s: 1, A: ('negative', 'low'):	0.032311
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.086666
  s: 2, A: ('negative', 'low'):	0.250918
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.041996
  s: 3, A: ('negative', 'high'):	0.020607
  s: 3, A: ('negative', 'low'):	0.391456
  s: 4, A: ('positive', 'high'):	0.002105
  s: 4, A: ('positive', 'low'):	0.030456
  s: 4, A: ('negative', 'high'):	0.001081


  s: 1, A: ('negative', 'low'):	0.017377
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.012677
  s: 2, A: ('negative', 'low'):	0.032375
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.003380
  s: 3, A: ('negative', 'high'):	0.000550
  s: 3, A: ('negative', 'low'):	0.016573
  s: 4, A: ('positive', 'high'):	0.163514
  s: 4, A: ('positive', 'low'):	0.475048
  s: 4, A: ('negative', 'high'):	0.019546
  s: 4, A: ('negative', 'low'):	0.196016
  s: 5, A: ('positive', 'high'):	0.016848
  s: 5, A: ('positive', 'low'):	0.007631
  s: 5, A: ('negative', 'high'):	0.000255
  s: 5, A: ('negative', 'low'):	0.000135
  total_prob: 1.000000
--- utterance: 5 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.150318
  s: 1, A: ('negative', 'low'):	0.009269
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
 

  s: 3, A: ('positive', 'low'):	0.032742
  s: 3, A: ('negative', 'high'):	0.012698
  s: 3, A: ('negative', 'low'):	0.051731
  s: 4, A: ('positive', 'high'):	0.038350
  s: 4, A: ('positive', 'low'):	0.064578
  s: 4, A: ('negative', 'high'):	0.002630
  s: 4, A: ('negative', 'low'):	0.006265
  s: 5, A: ('positive', 'high'):	0.015435
  s: 5, A: ('positive', 'low'):	0.004000
  s: 5, A: ('negative', 'high'):	0.000411
  s: 5, A: ('negative', 'low'):	0.000080
  total_prob: 1.000000
----------------------
  CONTEXT: WC83
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.558663
  s: 1, A: ('negative', 'low'):	0.094708
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.124639
  s: 2, A: ('negative', 'low'):	0.077000
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.000618
  s: 3, A: ('negative'

...

  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.021485
  s: 2, A: ('negative', 'low'):	0.062203
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.052579
  s: 3, A: ('negative', 'high'):	0.025800
  s: 3, A: ('negative', 'low'):	0.490104
  s: 4, A: ('positive', 'high'):	0.006377
  s: 4, A: ('positive', 'low'):	0.092272
  s: 4, A: ('negative', 'high'):	0.003274
  s: 4, A: ('negative', 'low'):	0.065446
  s: 5, A: ('positive', 'high'):	0.017439
  s: 5, A: ('positive', 'low'):	0.031196
  s: 5, A: ('negative', 'high'):	0.002170
  s: 5, A: ('negative', 'low'):	0.001302
  total_prob: 1.000000
--- utterance: 4 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.059365
  s: 1, A: ('negative', 'low'):	0.027094
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.015078
  s: 2, A: ('negative', 'low'):	0.038507
 

--- utterance: 3 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.132327
  s: 1, A: ('negative', 'low'):	0.068467
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.014077
  s: 2, A: ('negative', 'low'):	0.040755
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.039633
  s: 3, A: ('negative', 'high'):	0.019447
  s: 3, A: ('negative', 'low'):	0.369428
  s: 4, A: ('positive', 'high'):	0.008408
  s: 4, A: ('positive', 'low'):	0.121653
  s: 4, A: ('negative', 'high'):	0.004316
  s: 4, A: ('negative', 'low'):	0.086286
  s: 5, A: ('positive', 'high'):	0.031862
  s: 5, A: ('positive', 'low'):	0.056998
  s: 5, A: ('negative', 'high'):	0.003965
  s: 5, A: ('negative', 'low'):	0.002378
  total_prob: 1.000000
--- utterance: 4 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negativ

  s: 5, A: ('positive', 'high'):	0.051822
  s: 5, A: ('positive', 'low'):	0.092703
  s: 5, A: ('negative', 'high'):	0.006449
  s: 5, A: ('negative', 'low'):	0.003868
  total_prob: 1.000000
--- utterance: 4 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.046088
  s: 1, A: ('negative', 'low'):	0.021034
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.040653
  s: 2, A: ('negative', 'low'):	0.103821
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.000488
  s: 3, A: ('negative', 'high'):	0.000079
  s: 3, A: ('negative', 'low'):	0.002393
  s: 4, A: ('positive', 'high'):	0.100877
  s: 4, A: ('positive', 'low'):	0.293072
  s: 4, A: ('negative', 'high'):	0.012059
  s: 4, A: ('negative', 'low'):	0.120929
  s: 5, A: ('positive', 'high'):	0.175126
  s: 5, A: ('positive', 'low'):	0.079320
  s: 5, A: ('negative', 'high'):	0.002655


  s: 5, A: ('negative', 'low'):	0.003022
  total_prob: 1.000000
----------------------
  CONTEXT: WC69
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.253965
  s: 1, A: ('negative', 'low'):	0.043054
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.060461
  s: 2, A: ('negative', 'low'):	0.037352
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.003909
  s: 3, A: ('negative', 'high'):	0.019654
  s: 3, A: ('negative', 'low'):	0.143363
  s: 4, A: ('positive', 'high'):	0.076709
  s: 4, A: ('positive', 'low'):	0.036221
  s: 4, A: ('negative', 'high'):	0.019120
  s: 4, A: ('negative', 'low'):	0.081562
  s: 5, A: ('positive', 'high'):	0.199398
  s: 5, A: ('positive', 'low'):	0.011641
  s: 5, A: ('negative', 'high'):	0.012049
  s: 5, A: ('negative', 'low'):	0.001542
  total_prob: 1.000000

  s: 5, A: ('negative', 'low'):	0.000535
  total_prob: 1.000000
--- utterance: 2 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.081617
  s: 1, A: ('negative', 'low'):	0.018059
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.101305
  s: 2, A: ('negative', 'low'):	0.123412
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.017723
  s: 3, A: ('negative', 'high'):	0.021094
  s: 3, A: ('negative', 'low'):	0.308358
  s: 4, A: ('positive', 'high'):	0.041101
  s: 4, A: ('positive', 'low'):	0.108597
  s: 4, A: ('negative', 'high'):	0.013571
  s: 4, A: ('negative', 'low'):	0.116016
  s: 5, A: ('positive', 'high'):	0.034434
  s: 5, A: ('positive', 'low'):	0.011248
  s: 5, A: ('negative', 'high'):	0.002756
  s: 5, A: ('negative', 'low'):	0.000707
  total_prob: 1.000000
--- utterance: 3 ----
  s: 1, A: ('positive', 'high'):	0.0000

In [3]:
#example on marginalizing over the affect to get the probability of a state. This is done over every utterance/context combination
# for c in range(len(contexts)):
#     for u in output[contexts[c]].keys():
#         out = output[contexts[c]][u]
#         y = [sum(affect.values()) for affect in out.values()]
#         x = list(out.keys())

In [4]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10, 10))
labels = ["terrible", "bad", "neutral", "good", "amazing"]


for c in range(len(contexts)):
    for u in output[contexts[c]].keys():
        out = output[contexts[c]][u]
        y = [sum(affect.values()) for affect in out.values()]
        x = list(out.keys())

        ax1 = plt.subplot2grid((9,5),(c,u-1))
        ax1.set_ylim(0, 1)
        ax1.plot(list(out.keys()),y)
        plt.grid()
        plt.xticks(list(out.keys()), labels) 
        plt.yticks(np.arange(0.0, 1.0, 0.25))


fig.subplots_adjust(hspace=.5, wspace=.5)
plt.show()

IndexError: invalid index

In [5]:
#write output 
def ddict2dict(d):
    '''
    convert recursive defaultdict to dict
    '''
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = ddict2dict(v)
    return dict(d)

# for c in range(len(contexts)):
#     for u in output[contexts[c]].keys():
#         out = output[contexts[c]][u]
#         y = [sum(affect.values()) for affect in out.values()]
#         x = list(out.keys())

for c in range(len(contexts)):
    for u in output[contexts[c]].keys():
        out = output[contexts[c]][u]
        print([sum(affect.values()) for affect in out.values()])
        #output[contexts[c]][u] = [sum(affect.values()) for affect in out.values()]
        
output = ddict2dict(output)
with open('synthetic_fig_5_labels.json', 'w') as fp:
    fp.write(str(output))

[0.6885585125199486, 0.0895655728139647, 0.02998821214514498, 0.10933992274766968, 0.08254777977327174]
[0.31776479284356884, 0.3410619610461838, 0.05857621514855846, 0.20301786404427927, 0.07957916691740972]
[0.2542215468799798, 0.13996466840939026, 0.21207373027569237, 0.32474247522825617, 0.06899757920668127]
[0.1079704362176795, 0.056506144789582734, 0.03782713134471324, 0.6739025782741531, 0.12379370937387131]
[0.18430719157064107, 0.050080839864016416, 0.01817226058595307, 0.2799041694451007, 0.46753553853428875]
[0.6072534069790775, 0.06345904631207809, 0.15841525678658, 0.09503747444078535, 0.07583481548147883]
[0.25926933788574574, 0.2235640616174544, 0.28627538723035023, 0.16325506011391236, 0.06763615315253717]
[0.12530078564141867, 0.05542212603677869, 0.6261027846898208, 0.15774933943527, 0.035424964196711754]
[0.09204036750838218, 0.03869835381811384, 0.1931497802382945, 0.5661840383079286, 0.10992746012728077]
[0.16812087209566323, 0.03670069437009041, 0.0992900275874328